In [1]:
# Libraries and Constants

SERVER_IP = '192.168.27.99'
API_KEY   = 'whatever'

# Using Mistral with straight OpenAI

In [2]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url=F"http://{SERVER_IP}:1234/v1", api_key=API_KEY)

completion = client.chat.completions.create(
  model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q6_K.gguf",
  messages=[
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful. You are also a Python expert."},
    {"role": "user", "content": "Tell me some Python code that lists all numbers between 110 and 120."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" Here's a simple Python code that will list all the numbers between 110 and 120:\n```python\nfor i in range(110, 121):\n    print(i)\n```\nThis code uses a `for` loop with the `range()` function to iterate through all the integers from 110 (inclusive) to 121 (exclusive). The `print()` function is used to output each number to the console.", role='assistant', function_call=None, tool_calls=None)


In [ ]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url=F"http://{SERVER_IP}:1234/v1", api_key=API_KEY)

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    completion = client.chat.completions.create(
        model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q6_K.gguf",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    
    # Uncomment to see chat history
    # import json
    # gray_color = "\033[90m"
    # reset_color = "\033[0m"
    # print(f"{gray_color}\n{'-'*20} History dump {'-'*20}\n")
    # print(json.dumps(history, indent=2))
    # print(f"\n{'-'*55}\n{reset_color}")

    print()
    history.append({"role": "user", "content": input("> ")})

# Using Mistral with Autogen

In [9]:
import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

cache_seed = 42 # default seed is 41

my_models = [
    # mistral-7b-instruct-v0.1.Q6_K.gguf
    {
        "model": "TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf",
        "api_type": "open_ai",
        "api_base": F"http://{SERVER_IP}:1234/v1",
        "api_key":  API_KEY # "sk-XoMik5CKunt0hvEJH5QgT3B1bkFJeoQoffxGuvWBf3G3EFKn"
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "api_base": {
           f"http://{SERVER_IP}:1234/v1"
        }
    }
)

config_list

[{'model': 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf',
  'api_type': 'open_ai',
  'api_base': 'http://192.168.27.99:1234/v1',
  'api_key': 'whatever'}]

In [15]:
llm_config={
    # "request_timeout": 600,
    "seed": cache_seed,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0  # temperature for sampling
}

llm_config

{'seed': 42,
 'config_list': [{'model': 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf',
   'api_type': 'open_ai',
   'api_base': 'http://192.168.27.99:1234/v1',
   'api_key': 'whatever'}],
 'temperature': 0}

In [22]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    system_message = """Reply TERMINATE if the task has been completed""",
    
    llm_config = llm_config,
)

user_proxy

[autogen.oai.client: 03-30 15:30:48] {122} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [17]:
# create an AssistantAgent named "assistant"
import autogen

assistant = autogen.AssistantAgent(
    name="assistant",
    system_message = "You are a coder specializing in Python.",
    llm_config=llm_config
)

assistant

[autogen.oai.client: 03-30 13:14:03] {122} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [18]:
task = """
write a python method to output number 101 to 106
"""

In [19]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description

chat_res = user_proxy.initiate_chat(
    assistant,
    message=task,
    model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q6_K.gguf",
    messages=[
        {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful. You are also a Python expert."},
        {"role": "user", "content": "Tell me some Python code that lists all numbers between 110 and 120."}
    ],
)

user_proxy (to assistant):


write a python method to output number 101 to 106


--------------------------------------------------------------------------------
messages: [{'content': '\nwrite a python method to output number 101 to 106\n', 'role': 'user'}]
messages: [{'content': '\nwrite a python method to output number 101 to 106\n', 'role': 'user'}]
reply_func_tuple["config"]: None
messages: [{'content': '\nwrite a python method to output number 101 to 106\n', 'role': 'user'}]
reply_func_tuple["config"]: None
messages: [{'content': '\nwrite a python method to output number 101 to 106\n', 'role': 'user'}]
reply_func_tuple["config"]: None
messages: [{'content': '\nwrite a python method to output number 101 to 106\n', 'role': 'user'}]
reply_func_tuple["config"]: None


TypeError: Completions.create() got an unexpected keyword argument 'api_base'

In [ ]:
# autogen.Completion.clear_cache(seed=cache_seed)

In [ ]:
i=1
for cr in chat_res.chat_history:
    print(f"Step {i} --> {cr["content"].rstrip()[-9:]}")
    i=i+1

In [ ]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

In [ ]:
# followup of the previous question
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of their stock price change YTD. Save the data to stock_price_ytd.csv, and save the plot to stock_price_ytd.png.""",
)

In [ ]:
from IPython.display import display, Image

try:
    image = Image(filename="coding/stock_price_ytd.png")
    display(image)
except FileNotFoundError:
    print("Image not found. Please check the file name and modify if necessary.")